In [1]:
%reset -f

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext Cython

In [3]:
%reload_ext Cython

In [4]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [5]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
from data_functions import make_date_to_MYR, make_MYR_to_date
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )
#import data_process
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import models, layers
import warnings
warnings.filterwarnings('ignore')
from utils import statistics_first_month, p_val, stat_matrices 
#from path import col_seq_can_all, col_seq_can 
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [6]:
# #setup cython
# #for cython build if needed -> cd to /modules folder and then run
# %cd ..
# !python3 setup.py build_ext --inplace
# %cd ./causal_data_creation/canary_application/

In [7]:
%config Completer.use_jedi = False

### Load pre matching data

In [8]:
pp_str = 'Pre_Matching_DATA/'
Path("./"+pp_str).mkdir(exist_ok=True)

In [9]:
# %%time
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/CANARY_PAPER/DATA/Pre_Matching_DATA/treated2Past2.p ./Pre_Matching_DATA/treated2.p; 
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/CANARY_PAPER/DATA/Pre_Matching_DATA/controlsPast.p ./Pre_Matching_DATA/controls.p;
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/CANARY_PAPER/DATA/Pre_Matching_DATA/pmpm.p ./Pre_Matching_DATA/pmpm.p;
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/CANARY_PAPER/DATA/tempHybridSelectionMoreData/random.p ./Pre_Matching_DATA/random.p;

In [10]:
treated = pd.read_pickle(pp_str+'treated2Past2.p')
controls = pd.read_pickle(pp_str+'controlsPast.p')
controls2  = controls[controls['HasPast'] == 1].reset_index(drop = True)
pmpm =pd.read_pickle(pp_str+'pmpm.p')
random_controls = pd.read_pickle(pp_str+'random.p')

In [11]:
pmpm.MBR_GNDR.replace({"F":0,"M":1}, inplace=True)

In [12]:
treated.shape, treated.MYR.min(), treated.MYR.max()

((22364, 92), 201710, 202107)

In [13]:
controls.shape, controls.MYR.min(), controls.MYR.max()

((9582196, 92), 201701, 202107)

In [14]:
pmpm.shape, pmpm.MYR.min(), pmpm.MYR.max()

((9650961, 91), 201701, 202107)

### Stats for all months

In [15]:
#All month - controls with both HasPast=0 and 1
conA = controls.describe().loc[['mean', 'std', 'count']].T
conA['count'] = conA['count'].astype(int)

In [16]:
#All month - controls with only HasPast=1
conHA = controls2.describe().loc[['mean', 'std', 'count']].T
conHA['count'] = conHA['count'].astype(int)

In [19]:
controls_slice = controls[(controls.MYR>=treated.MYR.min())&(controls.MYR<=treated.MYR.max())]

In [20]:
controls_slice_haspast = controls[(controls.MYR>=treated.MYR.min())&(controls.MYR<=treated.MYR.max())&(controls.HasPast==1)]

In [23]:
#All month - controls in treated time range with both HasPast=0 and 1
conA_slice = controls_slice.describe().loc[['mean', 'std', 'count']].T
conA_slice['count'] = conA_slice['count'].astype(int)

In [24]:
#All month - controls in treated time range with only HasPast=1
conHA_slice = controls_slice_haspast.describe().loc[['mean', 'std', 'count']].T
conHA_slice['count'] = conHA_slice['count'].astype(int)

In [29]:
conA.to_csv("./unmatched_control_stats/AllControls_AllMonthStat.csv", index=True, header=True)
conHA.to_csv("./unmatched_control_stats/HasPastControls_AllMonthStat.csv", index=True, header=True)
conA_slice.to_csv("./unmatched_control_stats/AllControlsinTreatedMonths_AllMonthStat.csv", index=True, header=True)
conHA_slice.to_csv("./unmatched_control_stats/HasPastControlsinTreatedMonths_AllMonthStat.csv", index=True, header=True)

### Stats for first month records only for each patient

In [30]:
all_columns = list(set(col for col in pmpm.columns)-set(['PERS_ID', 'MYR','MEDICAID_LOB']))

In [31]:
#First month - controls with both HasPast=0 and 1
conA = statistics_first_month(controls, all_columns)

In [32]:
#First month - controls with only HasPast=1
conHA = statistics_first_month(controls2, all_columns)

In [33]:
controls_slice = controls[(controls.MYR>=treated.MYR.min())&(controls.MYR<=treated.MYR.max())]

In [34]:
controls_slice_haspast = controls[(controls.MYR>=treated.MYR.min())&(controls.MYR<=treated.MYR.max())&(controls.HasPast==1)]

In [35]:
#First month - controls in treated time range with both HasPast=0 and 1
conA_slice = statistics_first_month(controls_slice, all_columns)

In [36]:
#First month - controls in treated time range with only HasPast=1
conHA_slice = statistics_first_month(controls_slice_haspast, all_columns)

In [41]:
conA.to_csv("./unmatched_control_stats/AllControls_FirstMonthStat.csv", index=True, header=True)
conHA.to_csv("./unmatched_control_stats/HasPastControls_FirstMonthStat.csv", index=True, header=True)
conA_slice.to_csv("./unmatched_control_stats/AllControlsinTreatedMonths_FirstMonthStat.csv", index=True, header=True)
conHA_slice.to_csv("./unmatched_control_stats/HasPastControlsinTreatedMonths_FirstMonthStat.csv", index=True, header=True)